In [1]:
import pandas as pd
import os
import glob
import re 

In [2]:
# Since the collider names are too detailed, here we create a dictionary with patterns to classify them into our categories of interest

patterns = {'\d{2}_Sa':'Passive_Agent', '\d{2}_Cma':'Active_Agent', 'Building_\d+': 'Building'}
patterns.update(dict.fromkeys(['Castle-TaskBuilding_56', 'Crane_59','HighSilo-TaskBuilding_49', 'Windmill-TaskBuilding_10_1', 'Church-TaskBuilding_16'], 'Global_Landmark'))
patterns.update(dict.fromkeys(['^TaskBuilding_2$','^TaskBuilding_3$', '^TaskBuilding_5$', '^TaskBuilding_8$', '^TaskBuilding_9$', '^TaskBuilding_11$', '^TaskBuilding_13$', '^TaskBuilding_14$', '^TaskBuilding_20$', 
                               '^TaskBuilding_21$', '^TaskBuilding_23$','^TaskBuilding_27$', '^TaskBuilding_29$', '^TaskBuilding_32$', '^TaskBuilding_34$',  '^TaskBuilding_38$', '^TaskBuilding_41$', '^TaskBuilding_42$', 
                               '^TaskBuilding_44$', '^TaskBuilding_45$', '^TaskBuilding_47$', '^TaskBuilding_50$', '^TaskBuilding_51$', '^TaskBuilding_52$', 'BasketballCourt_58', 'Construction_57', 
                               '^Graffity_02$', '^Graffity_03$', '^Graffity_05$', '^Graffity_08$', '^Graffity_09$', '^Graffity_11$', '^Graffity_13$', '^Graffity_14$', '^Graffity_20$', 
                               '^Graffity_21$', '^Graffity_23$', '^Graffity_27$', '^Graffity_29$', '^Graffity_32$', '^Graffity_34$', '^Graffity_38$', '^Graffity_41$', '^Graffity_42$', 
                               '^Graffity_44$', '^Graffity_45$', '^Graffity_47$',  '^Graffity_50$', '^Graffity_51$', '^Graffity_52$'], 'TaskBuilding_Public'))

patterns.update(dict.fromkeys(['^TaskBuilding_1$','^TaskBuilding_4$', '^TaskBuilding_6$', '^TaskBuilding_7$', '^TaskBuilding_12$', '^TaskBuilding_15$', '^TaskBuilding_17$', '^TaskBuilding_18$', '^TaskBuilding_19$', 
                               '^TaskBuilding_22$', '^TaskBuilding_24$','^TaskBuilding_25$', '^TaskBuilding_26$', '^TaskBuilding_28$', '^TaskBuilding_30$',  '^TaskBuilding_31$', '^TaskBuilding_33$', '^TaskBuilding_35$', 
                               '^TaskBuilding_36$', '^TaskBuilding_37$', '^TaskBuilding_39$', '^TaskBuilding_40$', '^TaskBuilding_43$', '^TaskBuilding_48$', '^TaskBuilding_54$','^TaskBuilding_55$',
                               '^Graffity_01$','^Graffity_04$', '^Graffity_06$', '^Graffity_07$', '^Graffity_12$', '^Graffity_15$', '^Graffity_17$', '^Graffity_18$', '^Graffity_19$', '^Graffity_22$', 
                               '^Graffity_24$','^Graffity_25$', '^Graffity_26$', '^Graffity_28$', '^Graffity_30$',  '^Graffity_31$', '^Graffity_33$', '^Graffity_35$', '^Graffity_36$', '^Graffity_37$', '^Graffity_39$', 
                               '^Graffity_40$', '^Graffity_43$', '^Graffity_48$', '^Graffity_54$', '^Graffity_55$' ], 'TaskBuilding_Residential'))
default_val = 'Background'

In [3]:
# Define the directory where individual entropy results are saved
output_entropy_dir = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/Window/entropy_results/Chao_Shen/"
entropy_log_path = "/Volumes/TwoTeras/0_Experiment_1/Entropy_Results/entropy_log_Window_reduced.csv"

# Get list of all entropy result CSV files
entropy_files = glob.glob(os.path.join(output_entropy_dir, "*.csv"))

# Initialize a list to collect DataFrames
dfs = []

# Loop through each file and append its DataFrame with additional columns
for file in entropy_files:
    try:
        # Extract Participant_ID and Session from the file name (assuming format participantID_Session.csv)
        filename = os.path.basename(file)
        participant_id, session = filename.split("_")[2], filename.split("_")[3].split(".")[0]
        
        # Read the file
        df = pd.read_csv(file)

        # Add Participant_ID and Session columns
        df['Participant_ID'] = participant_id
        df['Session'] = session

        # Append to the list of DataFrames
        dfs.append(df)
    except Exception as e:
        print(f"Error reading file {file}: {e}")

# Concatenate all DataFrames into one
if dfs:
    general_entropy_df = pd.concat(dfs, ignore_index=True)

    # Create the new column Agent_ID
    general_entropy_df['Agent_ID'] = general_entropy_df['Collider_Name'].str[:2]
    # Apply the patterns to classify Collider_CategoricalN
    #general_entropy_df['Category'] = general_entropy_df['Collider_Name'].apply( lambda x: next((val for key, val in patterns.items() if re.match(key, x)), default_val))
    
    # Create the new column Agent_Type
    general_entropy_df['Agent_Type'] = general_entropy_df['Agent_ID'].astype(int).apply(lambda x: 'Active' if x < 28 else 'Passive')
    general_entropy_df['Experiment'] = 1

    # Save the concatenated DataFrame
    general_entropy_df.to_csv(entropy_log_path, index=False)
    print(f"General entropy log saved to {entropy_log_path}")
else:
    print("No entropy result files found or unable to concatenate.")


General entropy log saved to /Volumes/TwoTeras/0_Experiment_1/Entropy_Results/entropy_log_Window_reduced.csv


In [4]:
len(general_entropy_df[general_entropy_df.Chao_Shen_Overall_Transition_Entropy  > 1])

8

In [5]:
len(general_entropy_df)

4923

In [6]:
general_entropy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4923 entries, 0 to 4922
Data columns (total 17 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Trial_ID                                      4923 non-null   object 
 1   Collider_Name                                 4923 non-null   object 
 2   Occurrence_Order                              4923 non-null   int64  
 3   Gaze_Sequence_Length                          4923 non-null   int64  
 4   Chao_Shen_Overall_Transition_Entropy          4923 non-null   float64
 5   Window_Start                                  4923 non-null   object 
 6   Window_End                                    4923 non-null   object 
 7   Chao_Shen_Transition_Entropy_Agent            4923 non-null   float64
 8   Chao_Shen_Transition_Entropy_Task_Building    4753 non-null   float64
 9   Chao_Shen_Transition_Entropy_Background       4907 non-null   f

In [7]:
# Create the new column Agent_ID
general_entropy_df['Agent_ID'] = general_entropy_df['Collider_Name'].str[:2]

In [8]:
general_entropy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4923 entries, 0 to 4922
Data columns (total 17 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Trial_ID                                      4923 non-null   object 
 1   Collider_Name                                 4923 non-null   object 
 2   Occurrence_Order                              4923 non-null   int64  
 3   Gaze_Sequence_Length                          4923 non-null   int64  
 4   Chao_Shen_Overall_Transition_Entropy          4923 non-null   float64
 5   Window_Start                                  4923 non-null   object 
 6   Window_End                                    4923 non-null   object 
 7   Chao_Shen_Transition_Entropy_Agent            4923 non-null   float64
 8   Chao_Shen_Transition_Entropy_Task_Building    4753 non-null   float64
 9   Chao_Shen_Transition_Entropy_Background       4907 non-null   f